# அமர்வு 1 – Foundry Local (சாட் பூட்ஸ்ட்ராப்)

இந்த நோட்புக் Foundry Local-ஐ பூட்ஸ்ட்ராப் செய்கிறது, விருப்பமான மாடல் அலியாஸ்-ஐ பதிவிறக்குகிறது, மேலும் ஒரு நிலையான மற்றும் ஸ்ட்ரீமிங் சாட் நிறைவு செயல்பாட்டை மேற்கொள்கிறது.


# சூழல்
இந்த அமர்வில் Foundry Local மூலம் ஒரு சிறிய உள்ளூர் மொழி மாதிரியை பதிலளிக்கச் செய்வதற்கான அடிப்படை வழிமுறைகள் அறிமுகப்படுத்தப்படும். நீங்கள் செய்ய வேண்டியது:
- SDK / கிளையன்ட் சார்ந்த பொருத்தங்களை நிறுவுதல்.
- தேர்ந்தெடுக்கப்பட்ட பெயருக்கு Foundry Local மேலாளரை தொடங்குதல் (இயல்புநிலை: `phi-3.5-mini`).
- மாதிரி மெட்டாடேட்டாவில் விருப்ப புலங்களை ஏற்கும் வகையில் பாதுகாப்பான monkey‑patch ஐ பயன்படுத்துதல்.
- ஒரு நிலையான உரையாடல் முடிவு கோரிக்கையை அனுப்புதல்.
- பதிலின் டோக்கன்களை ஒவ்வொன்றாக ஸ்ட்ரீம் செய்தல்.

இக்குறிக்கோள் உங்கள் உள்ளூர் ரன்டைம் மற்றும் நெட்வொர்க் பாதையை RAG, வழிமுறைகள், அல்லது முகவரிகள் நோக்கி நகர்வதற்கு முன் சரிபார்ப்பது.


### விளக்கம்: சார்பு நிறுவல்
இந்த குறைந்தபட்ச உரையாடல் ஓட்டத்திற்குத் தேவையான Python தொகுப்புகளை நிறுவுகிறது:
- `foundry-local-sdk`: உள்ளூர் மாதிரிகள் மற்றும் சேவை வாழ்க்கைச்சுழற்சியை நிர்வகிக்க.
- `openai`: உரையாடல் முடிவுகளுக்கான பரிச்சயமான கிளையன்ட் அப்ஸ்ட்ராக்ஷன்.
- `rich`: நோட்புக் வெளியீட்டிற்கான அழகான அச்சிடல்.

மீண்டும் இயக்குவது பாதுகாப்பானது (idempotent). உங்கள் சூழலில் ஏற்கனவே இவை இருந்தால் தவிர்க்கவும்.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### விளக்கம்: முக்கிய இறக்குமதிகள்
நோட்ட்புக் முழுவதும் பயன்படுத்தப்படும் மாட்யூல்களை கொண்டு வருகிறது:
- `FoundryLocalManager` உள்ளூர் மாடல் ரன்டைமுடன் தொடர்பு கொள்ள.
- `OpenAI` கிளையண்ட், பரிச்சயமான சாட் கம்ப்ளீஷன் API மேற்பரப்பை மீண்டும் பயன்படுத்துவதற்காக.
- `rich.print` அழகான வெளியீட்டிற்காக.

இங்கு எந்த நெட்வொர்க் அழைப்புகளும் நடக்கவில்லை—இது வெறும் நேம்ஸ்பேஸை தயாரிக்கிறது.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### விளக்கம்: மேலாளர் தொடக்கம் & மெட்டாடேட்டா திருத்தம்
தேர்ந்தெடுக்கப்பட்ட alias க்கான `FoundryLocalManager` ஐ தொடங்குகிறது மற்றும் `promptTemplate` `null` ஆக இருக்கக்கூடிய சேவை பதில்களை நன்கு கையாள ஒரு பாதுகாப்பான monkey‑patch ஐ பயன்படுத்துகிறது.

முக்கிய முடிவுகள்:
- சேவை நிலை மற்றும் இறுதிப்புள்ளியை உறுதிப்படுத்துகிறது.
- காசேடு செய்யப்பட்ட மாதிரிகளை பட்டியலிடுகிறது (உள்ளூர் சேமிப்பகத்தை சரிபார்க்கிறது).
- alias க்கான உறுதியான மாதிரி ID ஐ தீர்மானிக்கிறது (பின்னர் உரையாடல் அழைப்புகளில் பயன்படுத்தப்படுகிறது).

உங்கள் சேவை மெட்டாடேட்டாவில் சரிபார்ப்பு பிரச்சினைகளை சந்தித்தால், SDK ஐ பிரிக்காமல் சுத்தம் செய்ய இந்த முறை எப்படி உதவுகிறது என்பதை இது காட்டுகிறது.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### விளக்கம்: அடிப்படை உரையாடல் நிறைவு
உள்ளூர் Foundry இறுதிப்புள்ளியை நோக்கி `OpenAI`-இன் இணக்கமான கிளையண்டை உருவாக்கி, ஒரு ஒற்றை non‑streaming உரையாடல் நிறைவை செயல்படுத்துகிறது. இங்கு கவனம் செலுத்தவும்:
- மாடல் பிழையின்றி பதிலளிக்க வேண்டும்.
- தாமதம் / வெளியீட்டு வடிவத்தை சரிபார்க்கவும்.
- வளங்களை பாதுகாக்க `max_tokens`-ஐ மிதமாக வைத்திருக்கவும்.

இது தோல்வியடைந்தால், Foundry Local சேவை இயங்குகிறதா மற்றும் alias சரியாக தீர்க்கப்படுகிறதா என்பதை மீண்டும் சரிபார்க்கவும்.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### விளக்கம்: ஸ்ட்ரீமிங் சொடுகை நிறைவு
சிறந்த உணரப்பட்ட தாமதம் மற்றும் தொடர்பு UX க்காக டோக்கன் ஸ்ட்ரீமிங்கை விளக்குகிறது. வரிசை வந்தடையும் போது குறுக்கீடுகளை அடுத்தடுத்து அச்சிடுகிறது:
- ஆரம்பகால பகுதி வெளியீடு முக்கியமானது என்ற இடங்களில் உரையாடல் UI களுக்கு பயனுள்ளதாக உள்ளது.
- முழு நிறைவு தாமதத்துக்கு எதிராக டோக்கன் ஊடுருவல் அளவை அளவிட அனுமதிக்கிறது.

இந்த முறைமையை டோக்கன்களை சேர்க்க, முன்னேற்றம் விட்ஜெட்டை புதுப்பிக்க, அல்லது நடுவில் உருவாக்கத்தை நிறுத்த மாற்றலாம்.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
